In [1]:
#!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

In [2]:
#!kaggle competitions download -c rsna-pneumonia-detection-challenge

In [3]:
#!unzip rsna-pneumonia-detection-challenge.zip -d rsna/

In [50]:
import pandas as pd
import pydicom 
import os
import cv2
from sklearn.model_selection import train_test_split
import pathlib
import tqdm
import shutil

In [30]:
covid_data_path = './covid-chestxray-dataset/'
metadata_path =  covid_data_path+ 'metadata.csv'

In [6]:
covid_df = pd.read_csv(metadata_path)


In [7]:
covid_df.head()

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,needed_supplemental_O2,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 27
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,infiltrate in the upper lobe of the left lung,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [8]:
covid_PA_cxrs_df = (covid_df['view'] == 'PA')
covid_patients_df = (covid_df['finding'] == 'COVID-19')

In [9]:
covid_PA_cxrs_df

0       True
1       True
2       True
3       True
4       True
       ...  
324     True
325     True
326     True
327     True
328    False
Name: view, Length: 329, dtype: bool

In [10]:
covid_df.shape

(329, 28)

In [11]:
rsnam_data_path = './rsna/'
rsna_metadata_path = rsnam_data_path + 'stage_2_train_labels.csv'


In [12]:
rsna_df = pd.read_csv(rsna_metadata_path)

In [13]:
rsna_df.head()

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1


In [14]:
rsna_normal_df = rsna_df[rsna_df['Target'] == 0]
rsna_pneum_df = rsna_df[rsna_df['Target'] == 1]

In [15]:
rsna_normal_df.shape

(20672, 6)

In [16]:
rsna_pneum_df.shape

(9555, 6)

In [17]:
num_rsna_imgs = 907

In [18]:
 # Convert dicom files of CXRs with no findings to jpg if not done already in a previous run. Select only PA views.
pa_file_counter = 0
pa_normal_idxs = []
for df_idx in rsna_normal_df.index.values.tolist():
    filename = rsna_normal_df.loc[df_idx]['patientId']
    ds = pydicom.dcmread(os.path.join(rsnam_data_path + 'stage_2_train_images/' + filename + '.dcm'))
    if ds.SeriesDescription.split(' ')[1] == 'PA':
        pixel_arr = ds.pixel_array
        cv2.imwrite(os.path.join(rsnam_data_path + filename + '.jpg'), pixel_arr)
        pa_normal_idxs.append(df_idx)
        pa_file_counter += 1
    if pa_file_counter >= num_rsna_imgs // 2:
        break
rsna_normal_df = rsna_normal_df.loc[pa_normal_idxs]

In [19]:
rsna_pneum_df

,patientId,x,y,width,height,Target
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1
14,00aecb01-a116-45a2-956c-08d2fa55433f,288.0,322.0,94.0,135.0,1
...,...,...,...,...,...,...
30220,c1e73a4e-7afe-4ec5-8af6-ce8315d7a2f2,316.0,504.0,179.0,273.0,1
30221,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,609.0,464.0,240.0,284.0,1
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [20]:
pa_file_counter

453

In [21]:
# Convert dicom files of CXRs with pneumonia to jpg if not done already in a previous run. Select only PA views.
pa_file_counter = 0
pa_pneum_idxs = []
num_remaining = num_rsna_imgs - num_rsna_imgs // 2
for df_idx in rsna_pneum_df.index.values.tolist():
    filename = rsna_pneum_df.loc[df_idx]['patientId']
    ds = pydicom.dcmread(os.path.join(rsnam_data_path + 'stage_2_train_images/' + filename + '.dcm'))
    if ds.SeriesDescription.split(' ')[1] == 'PA':
        pixel_arr = ds.pixel_array
        cv2.imwrite(os.path.join(rsnam_data_path + filename + '.jpg'), pixel_arr)
        pa_pneum_idxs.append(df_idx)
        pa_file_counter += 1
    if pa_file_counter >= num_remaining:
        break
rsna_pneum_df = rsna_pneum_df.loc[pa_pneum_idxs]

In [23]:
n_classes = 3

In [24]:
 classes = ['normal', 'COVID-19', 'other_pneumonia']

In [25]:
class_dict ={classes[i]: i for i in range(n_classes)}

In [26]:
class_dict

{'normal': 0, 'COVID-19': 1, 'other_pneumonia': 2}

In [27]:
label_dict = {i: classes[i] for i in range(n_classes)}  

In [28]:
label_dict

{0: 'normal', 1: 'COVID-19', 2: 'other_pneumonia'}

In [31]:
PA_covid_df = covid_df[covid_patients_df & covid_PA_cxrs_df]  # PA images diagnosed COVID
PA_covid_df['label'] = class_dict['COVID-19']
PA_pneum_df = covid_df[covid_df['finding'].isin(['SARS', 'Steptococcus']) & covid_PA_cxrs_df]   # PA images diagnosed with SARS
PA_pneum_df['label'] = class_dict['other_pneumonia']                 # Classify SARS with other viral pneumonias
file_df = pd.concat([PA_covid_df[['filename', 'label']], PA_pneum_df[['filename', 'label']]], axis=0)
file_df['filename'] = covid_data_path + 'images/' + file_df['filename'].astype(str)  # Set as absolute paths

# Organize some files from RSNA dataset into "normal", and "pneumonia" XRs
rsna_normal_filenames = rsnam_data_path + rsna_normal_df['patientId'].astype(str) + '.jpg'
rsna_pneum_filenames = rsnam_data_path + rsna_pneum_df['patientId'].astype(str) + '.jpg'
rsna_normal_file_df = pd.DataFrame({'filename': rsna_normal_filenames, 'label': class_dict['normal']})
rsna_pneum_file_df = pd.DataFrame({'filename': rsna_pneum_filenames, 'label': class_dict['other_pneumonia']})
rsna_file_df = pd.concat([rsna_normal_file_df, rsna_pneum_file_df], axis=0)

file_df = pd.concat([file_df, rsna_file_df], axis=0)  # Combine both datasets


<ipython-input-31-1c350423560e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PA_covid_df['label'] = class_dict['COVID-19']
<ipython-input-31-1c350423560e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PA_pneum_df['label'] = class_dict['other_pneumonia']                 # Classify SARS with other viral pneumonias


In [32]:
file_df

,filename,label
0,./covid-chestxray-dataset/images/auntminnie-a-...,1
1,./covid-chestxray-dataset/images/auntminnie-b-...,1
2,./covid-chestxray-dataset/images/auntminnie-c-...,1
3,./covid-chestxray-dataset/images/auntminnie-d-...,1
4,./covid-chestxray-dataset/images/nejmc2001573_...,1
...,...,...
4231,./rsna/3d7fb4b4-5e62-4591-aa4d-cbbc1ca3ffb7.jpg,2
4232,./rsna/3d7fb4b4-5e62-4591-aa4d-cbbc1ca3ffb7.jpg,2
4235,./rsna/3d895c52-0627-42b3-ba67-41a08f387da0.jpg,2
4236,./rsna/3d895c52-0627-42b3-ba67-41a08f387da0.jpg,2


In [33]:
def remove_text(img):
    '''
    Attempts to remove textual artifacts from X-ray images. For example, many images indicate the right side of the
    body with a white 'R'. Works only for very bright text.
    :param img: Numpy array of image
    :return: Array of image with (ideally) any characters removed and inpainted
    '''
    mask = cv2.threshold(img, 230, 255, cv2.THRESH_BINARY)[1][:, :, 0].astype(np.uint8)
    img = img.astype(np.uint8)
    result = cv2.inpaint(img, mask, 10, cv2.INPAINT_NS).astype(np.float32)
    return result

In [34]:
val_split = 0.1

In [37]:
file_df_train, file_df_test = train_test_split(file_df, test_size=val_split, stratify=file_df['label'])

In [39]:
file_df_test

,filename,label
3923,./rsna/3b8d4df3-3950-412b-9206-1cbb57c7943e.jpg,2
613,./rsna/0902e0fc-fcf3-40b5-ac04-c3ecc14c2355.jpg,2
1082,./rsna/0c9be296-285f-44dc-8599-afde00848e9d.jpg,0
3603,./rsna/39693e31-b11e-4cd5-a626-2862cb0ede1d.jpg,2
2700,./rsna/339d0607-e869-4869-ac6d-379172682713.jpg,2
...,...,...
975,./rsna/0bcfe7ee-ada5-48de-990b-2a2fd6c6ed15.jpg,2
268,./rsna/06737bed-8dc1-47d9-8c30-324282dcc84d.jpg,2
3815,./rsna/3adf7f71-3ded-406d-a08c-a61d152f78dc.jpg,2
2908,./rsna/34e19eb5-78d1-42b9-bc47-b043d97feefb.jpg,2


In [40]:
dest_dir = os.path.join(os.getcwd(), 'data')


In [41]:
dest_dir

'/lusnlsas/ramkik_data/covid19/covid/data'

In [45]:
print('Deleting old sets.')
for path in pathlib.Path(os.path.join(dest_dir, 'train/')).glob('*'):
    path.unlink()
for path in pathlib.Path(os.path.join(dest_dir, 'val/')).glob('*'):
    path.unlink()
for path in pathlib.Path(os.path.join(dest_dir, 'test/')).glob('*'):
    path.unlink()

Deleting old sets.


In [55]:
 # Copy images to appropriate directories
print('Copying training set images.')
for file_path in file_df_train['filename'].tolist():
    shutil.copy(file_path, os.path.join(dest_dir, 'train/'))
print('Copying validation set images.')
for file_path in file_df_test['filename'].tolist():
    shutil.copy(file_path, os.path.join(dest_dir, 'test/'))
# print('Copying test set images.')
# for file_path in tqdm(file_df_test['filename'].tolist()):
#     shutil.copy(file_path, os.path.join(dest_dir, 'test/'))

Copying training set images.
Copying validation set images.


In [58]:
!ls -lrt data/train | wc -l

848


In [59]:
file_df_train['filename']

1038    ./rsna/0c2e9b99-9a8f-4b44-854e-acd181a0208c.jpg
2922    ./rsna/34fdff09-5bc2-4df5-b8cf-3c37662037c8.jpg
1174    ./rsna/0ebc8268-df3d-45d8-8ee7-b34880c62830.jpg
341     ./rsna/06f1d0a2-d8c5-4229-9944-59da85c96b81.jpg
280     ./rsna/06951c33-b247-4daf-a087-cc082f83238b.jpg
                             ...                       
1200    ./rsna/0f8c91da-7e03-480e-8760-1604b1d53c97.jpg
1151    ./rsna/0dbb83c1-2214-4152-ac69-d1e7e25453cb.jpg
499     ./rsna/081e308c-0134-4ba3-b745-f632e37a83a1.jpg
663     ./rsna/095d6b7c-fa53-4f06-90b9-5c5f76038f04.jpg
919     ./rsna/0b6a7730-a747-42ce-ac00-f3dcc34df307.jpg
Name: filename, Length: 945, dtype: object

In [60]:
file_df_train['filename'] = file_df_train['filename'].str.split('/').str[-1]
file_df_test['filename'] = file_df_test['filename'].str.split('/').str[-1]

<ipython-input-60-9b9613865c60>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df_train['filename'] = file_df_train['filename'].str.split('/').str[-1]
<ipython-input-60-9b9613865c60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df_test['filename'] = file_df_test['filename'].str.split('/').str[-1]


In [62]:
file_df_train

,filename,label
1038,0c2e9b99-9a8f-4b44-854e-acd181a0208c.jpg,0
2922,34fdff09-5bc2-4df5-b8cf-3c37662037c8.jpg,2
1174,0ebc8268-df3d-45d8-8ee7-b34880c62830.jpg,2
341,06f1d0a2-d8c5-4229-9944-59da85c96b81.jpg,0
280,06951c33-b247-4daf-a087-cc082f83238b.jpg,0
...,...,...
1200,0f8c91da-7e03-480e-8760-1604b1d53c97.jpg,0
1151,0dbb83c1-2214-4152-ac69-d1e7e25453cb.jpg,0
499,081e308c-0134-4ba3-b745-f632e37a83a1.jpg,0
663,095d6b7c-fa53-4f06-90b9-5c5f76038f04.jpg,2


In [63]:
# Save training, validation and test sets
file_df_train.to_csv(dest_dir+'/train.csv')
file_df_test.to_csv(dest_dir+'/test.csv')